In [ ]:
!pip install python-geohash
!pip install pygeohash
!pip install fastdtw
!pip install numba
!pip install statsmodels
!pip install pyarrow

In [23]:
!pip install --upgrade pip setuptools wheel
!pip install --upgrade pyarrow --only-binary :all:

In [47]:

import imp
import pdb
from datetime import datetime
import pandas as pd

# Import our dependency
from read_data import *
from social_group import *
from dtw_fun import *
from network_filtering import *
from group_network import *
import warnings
warnings.filterwarnings("ignore")

# Data path
base_path = 'lee_county_monthly/lee_county_202307.csv'

# Define the starting month of the whole project. 2022-08-01 to 2023-07-31
# original_date=datetime(2022, 8, 1)

# Define the pre disaster date range
start_date = datetime(2023, 7, 1)

# 1. Filter Holiday records from 2019-12-24 18:00 to 22:00
start_time = pd.Timestamp("2022-12-24 18:00:00")
end_time   = pd.Timestamp("2022-12-24 22:00:00")

# Select the location you want
# somewhere in Maryland, Florida, NYC, Longisland, A small county in the middle of Long Island, Region around Brookhaven and Mastic Beach
selected_region = 'Lee_county'  


In [48]:
'''
1. Read Data
'''
# read_save_all_data(start_date, end_date, base_path)

filtered_df, min_lat, max_lat, min_lon, max_lon=read_region_data(selected_region, base_path)


'''
2. Filter local resident and residents with minimmun appearance in month
'''

linked_df = apply_residency_filter_appear(filtered_df, min_appearances=20)




Original DataFrame shape: (39169, 9)


In [49]:
'''
3. Identify Social Group
'''
user_group_df, result_df_with_group=user_group_links(start_date, linked_df)


    device_id            utc_timestamp_1            utc_timestamp_2  \
0  7376980707  2023-07-01T07:12:58-04:00  2023-07-01T08:08:41-04:00   
1  7376980707  2023-07-01T08:14:24-04:00  2023-07-01T08:24:19-04:00   
2  7376980707  2023-07-01T08:28:00-04:00  2023-07-01T08:48:37-04:00   
3  7376980707  2023-07-01T08:52:03-04:00  2023-07-01T09:09:59-04:00   
4  7376980707  2023-07-01T09:12:01-04:00  2023-07-01T09:32:06-04:00   

   latitude_1  longitude_1  geohash_1  latitude_2  longitude_2  geohash_2  \
0   26.514166   -81.936074  dhtwz6zbp   26.514374   -81.936425  dhtwz6z9q   
1   26.514374   -81.936425  dhtwz6z9q   26.513977   -81.936210  dhtwz6xz5   
2   26.513977   -81.936210  dhtwz6xz5   26.514008   -81.936737  dhtwz6xrr   
3   26.514008   -81.936737  dhtwz6xrr   26.513916   -81.936423  dhtwz6xwy   
4   26.513916   -81.936423  dhtwz6xwy   26.513977   -81.936420  dhtwz6xxp   

                 unix_time_1  ...          timestamp_1          timestamp_2  \
0  2023-07-01T07:12:58-04:00  .

In [50]:
'''
4. Identify Social Links using DTW
'''
df_dtw_results=dtw_compute(result_df_with_group,start_time, end_time, min_trj_len = 20)



Number of active users: 56744
Fast-DTW running …


100%|██████████| 841/841 [00:01<00:00, 475.35it/s]


Threshold selected: 0.6284386515617371
Number of candidate pairs predicted as social links: 169
Records during holiday: 0
Number of holiday bonding nodes area: 0
Number of holiday bonding user pairs (edges): 0
DTW Prediction details saved to Individual_social_network.csv


In [51]:
'''
5. Network Filtering
'''
graph_df=filter_network_eadm(df_dtw_results, linked_df, start_date)

Scanning groups: 100%|██████████| 16025/16025 [00:01<00:00, 10015.19it/s]


In [52]:
'''
6. Social Group Network
'''
group_network_df=build_group_network(graph_df, user_group_df, start_date)

Total individual links (predicted connected): 169
Number of group candidate pairs: 158
All Group 484
Unique Device Id count: 498
Group that have links: 156
